In [1]:
import requests
import json
import pandas as pd
from copy import deepcopy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from Preprocessing import Preprocessor

mypath = '/mnt/HARD/MinMax94/data/data_all/CSV/Raw_extended/'

## Station_config

In [2]:
stations_lmeteo = pd.read_csv('/mnt/HARD/MinMax94/data/data_all/CSV/stations_def.csv', index_col=0)
stations_rp5 = pd.read_csv('/mnt/HARD/MinMax94/data/data_all/CSV/stations_rp5.csv', index_col=0)

#stations_lmeteo = stations_lmeteo.merge(stations_rp5[['station_id', 'meteostation_id']], how='inner', on='station_id')

In [3]:
RUSSIAN_TIME_ZONES = {
    '2': 'USZ1',
    '3': 'MSK',
    '4': 'SAMT',
    '8': 'IRKT',
    '5': 'YEKT',
    '6': 'OMSK',
    '7': 'KRAT',
    '9': 'YAKT',
    '10': 'VLAT',
    '11': 'MAGT',
    '12': 'PETT'
}

In [4]:
stations = deepcopy(stations_lmeteo)
stations['station_type'] = "road"
stations['station_nature'] = "real"
stations['timezone'] = stations['timezone'].apply(lambda row: RUSSIAN_TIME_ZONES[str(row)])
stations['road_category'] = 1
stations['roadlayers'] = str({"1": {"type": "asphalt", "thickness": 0.2}})

In [5]:
z = stations[['station_id', 'timezone', 'longitude', 'latitude', 
              'station_type', 'station_nature', 'road_category',
              'roadlayers']]
z = z[z['station_id']==113]
x = {"station_config": z.to_dict(orient='records')[0]}

with open("file", "w") as file:
    json.dump(x, file)
    
with open("file", "r") as file:
    station_config = json.load(file)
station_config

{'station_config': {'latitude': 53.603,
  'longitude': 54.446999999999996,
  'road_category': 1,
  'roadlayers': "{'1': {'thickness': 0.2, 'type': 'asphalt'}}",
  'station_id': 113,
  'station_nature': 'real',
  'station_type': 'road',
  'timezone': 'MSK'}}

## Road_config

In [6]:
road_config = str({"road_config": {
    "maintainability_level": "low",
    "category": "1А"}})

## Global_forecast

In [7]:
from os import listdir
from os.path import isfile, join

wmo_id = 22867
global_forecast_path = '/mnt/HARD/MinMax94/data/data_all/CSV/forecast'
wmo_files = [f for f in listdir(global_forecast_path) if isfile(join(global_forecast_path, f))]
wmo_id_file = [f for f in wmo_files if f.startswith(str(wmo_id))][0]

forecast_data = pd.read_csv(global_forecast_path + '/' + wmo_id_file, sep=';', skiprows=6, index_col=False)
date_time_column = [col for col in forecast_data.columns if col.startswith('Местное время в')][0]
forecast_data.rename(columns={date_time_column: 'date_time_loc'}, inplace=True) 
forecast_data['wmo_id'] = wmo_id

### Get timezone from lat, lon

In [8]:
import datetime
import pytz
from tzwhere import tzwhere

def hours_from_utc(lat, lon):
    tzwh = tzwhere.tzwhere()
    timezone_str = tzwh.tzNameAt(lat, lon) # Seville coordinates
    timezone_str

    timezone_now = pytz.timezone(timezone_str)
    hours_from_utc = datetime.datetime.now(timezone_now).utcoffset().total_seconds()/60/60
    return hours_from_utc

lat, lon = 55.278888888889, 105.157222222222
print(hours_from_utc(lat, lon))

8.0


### Calculate distance between stations from lat, lon

In [9]:
import geopy.distance

def vincenty_dist(point_1, point_2):
    return geopy.distance.vincenty(point_1, point_2).km

coord_1 = (52.2296756, 21.0122287)
coord_2 = (52.406374, 16.9251681)

print(vincenty_dist(coord_1, coord_2), 'km')

279.35290160386563 km


### Load rp5 stations coordinates

In [10]:
import codecs

path = '/mnt/HARD/MinMax94/data/data_all/CSV/wmo_russia_list.txt'

with codecs.open(path, encoding='cp1251', errors='replace') as f:
    lines = f.readlines()
    f.close()

In [1]:
from selenium import webdriver
from selenium.webdriver.common import action_chains, keys
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

driver = webdriver.Chrome()
driver.get("https://rp5.ru/%D0%90%D1%80%D1%85%D0%B8%D0%B2_%D0%BF%D0%BE%D0%B3%D0%BE%D0%B4%D1%8B_%D0%B2_%D0%9C%D0%B0%D0%BB%D0%BE%D1%8F%D1%80%D0%BE%D1%81%D0%BB%D0%B0%D0%B2%D1%86%D0%B5")
action = action_chains.ActionChains(driver)

wmo_id = 22867
start = '01.01.2012'
end = '01.01.2017'

wmo_id_field = driver.find_element_by_id('wmo_id')
wmo_id_field.clear()
wmo_id_field.send_keys(str(wmo_id))
time.sleep(1)

action.send_keys(keys.Keys.ENTER)
action.perform()

download_archive_button = driver.find_element_by_id('tabSynopDLoad')
download_archive_button.click()

start_date_field = driver.find_element_by_id('calender_dload')
start_date_field.clear()
start_date_field.send_keys(start)

action.send_keys(keys.Keys.ENTER)
action.perform()

end_date_field = driver.find_element_by_id('calender_dload2')
end_date_field.clear()
end_date_field.send_keys(end)

driver.execute_script("document.getElementById('format2').click();")
driver.execute_script("document.getElementById('coding2').click();")

create_gz_button = driver.find_element_by_xpath("//*[contains(text(), 'Выбрать в файл GZ (архив)')]")
create_gz_button.click()

time.sleep(3)
download_ref = driver.find_element_by_link_text("Скачать")

wait = WebDriverWait(driver, 10)
try:
    wait.until(EC.element_to_be_clickable(download_ref))
    download_ref.click()
except:
    print('except')

except
